<a href="https://colab.research.google.com/github/aichaait/class-generator-1/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain langchain-community langchain-core langchain-text-splitters faiss-cpu langchain-ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
!curl -fsSL https://ollama.com/install.sh|sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!nohup ollama serve > ollama.log2>&1 &

nohup: redirecting stderr to stdout


In [ ]:
!ollama pull gemma

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling ef311de6af9d...   0% ▕▏    0 B/5.0 GB                  pulling manifest 
pulling ef311de6af9d...   0% ▕▏    0 B/5.0 GB                  pulling manifest 
pulling ef311de6af9d...   0% ▕▏    0 B/5.0 GB                  pulling manifest 
pulling ef311de6af9d...   0% ▕▏ 3.2 MB/5.0 GB                  pulling manifest 
pulling ef311de6af9d...   1% ▕▏  45 MB/5.0 GB                  pulling manifest 
pulling ef311de6af9d...   1% ▕▏  70 MB/5.0 GB                  pulling manifest 
pulling ef311de6af9d...   2% ▕▏ 125 MB/5.0 GB                  pulling manifest 
pulling ef311de6af9d...   3% ▕▏ 170 MB/5.0 GB                  pulling ma

In [ ]:
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="gemma")

In [ ]:
model.invoke("donner moi une anecdote")

"Un ancien joueur de football, après sa retraite, s'est installé dans un village pittoreste. Un jour, il a rencontré une vieille dame qui vivait dans une maison en ruine. La dame s'appelle Marie et elle vivait seule. Le joueur a appris qu'elle était une grande fan de football et qu'elle avait toujours rêvé de rencontrer un footballeur célèbre. Le joueur s'est présenté à Marie et lui a parlé de son expérience de footballeur. Marie était si heureuse qu'elle a immédiatement embrassé le joueur."

In [ ]:
from langchain_ollama import ChatOllama

model1 = ChatOllama(model="gemma")

In [ ]:
model1.invoke("donner moi une anecdote")

AIMessage(content='Un jour, un vieil homme s\'appuya contre un arbre et s\' demanda comment il avait pu vivre si longtemps. Il répondit qu\'il avait appris une règle importante : "Apprends à vivre pour toi-même, et la joie de vivre sera plus grande que la douleur de mourir."', additional_kwargs={}, response_metadata={'model': 'gemma', 'created_at': '2025-01-03T23:21:06.734112026Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1935598994, 'load_duration': 42584799, 'prompt_eval_count': 31, 'prompt_eval_duration': 5000000, 'eval_count': 62, 'eval_duration': 1886000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-496a142f-7bcd-48ac-961c-7a23de86bd73-0', usage_metadata={'input_tokens': 31, 'output_tokens': 62, 'total_tokens': 93})

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser

In [ ]:
chain.invoke("donner moi une anecdote")

'Un jour, un ancien étudiant de physique s\'apprêtait à soutenir sa famille. Un ami lui demanda combien de temps il faudrait travailler pour gagner la même quantité d\'argent qu\'il gagnait en étudiant. Le ancien étudiant répondit : "Je travaillerai environ trois fois plus longtemps, car dans l\'entreprise, on ne donne pas de points de gravité !".'

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_pdfs_from_folder(folder_path):
    all_pages = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            file_path = os.path.join(folder_path, filename)
            loader = PyPDFLoader(file_path)
            pages = loader.load_and_split()
            all_pages.extend(pages)
    return all_pages

def split_into_chunks(pages, chunk_size=1000, overlap=200):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        length_function=len
    )
    split_pages = []
    for page in pages:
        content = page.page_content
        split_pages.extend(splitter.split_text(content))
    return split_pages

In [ ]:
folder_path = "/content/"

In [ ]:
pages = load_pdfs_from_folder(folder_path)

In [ ]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 8.1 MB/s eta 0:00:00


In [ ]:
chunks = split_into_chunks(pages)
len(chunks)

206

In [ ]:
!ollama pull llama2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏  13 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   1% ▕▏  30 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   2% ▕▏  69 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   3% ▕▏ 110 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   3% ▕▏ 125 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   4% ▕▏ 157 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   5% ▕▏ 180 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings()

<ipython-input-32-de1c80212185>:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings()


In [ ]:
embedded_chunks = embeddings.embed_documents(chunks)

In [ ]:
len(embedded_chunks[1])

4096

In [ ]:
embedded_chunks[1]

[-3.2922887802124023,
 -1.4332776069641113,
 0.4308142066001892,
 0.27945104241371155,
 -0.3860175311565399,
 0.6841953992843628,
 0.9924793839454651,
 -0.6393488049507141,
 0.48531731963157654,
 -0.8258143663406372,
 0.34242409467697144,
 0.34257471561431885,
 0.6710636615753174,
 0.7785807847976685,
 -2.919830322265625,
 1.15532386302948,
 0.657168984413147,
 -0.8426532745361328,
 -0.5353966355323792,
 -1.8723276853561401,
 -1.146308183670044,
 0.3513626754283905,
 -0.15285427868366241,
 -1.2193979024887085,
 1.3119423389434814,
 -0.0525299571454525,
 -0.37898117303848267,
 -0.4082757830619812,
 -0.3185660243034363,
 -1.0684430599212646,
 -0.6206943988800049,
 -1.8883074522018433,
 0.7638101577758789,
 4.223773002624512,
 2.2037389278411865,
 -1.5539454221725464,
 -0.478209912776947,
 -0.34931680560112,
 1.084330439567566,
 1.7225494384765625,
 1.6725996732711792,
 -2.770491361618042,
 1.6694039106369019,
 0.0369953028857708,
 -0.5275623798370361,
 0.14994513988494873,
 -0.1688986569

In [ ]:
from langchain.vectorstores import FAISS


vectorstore = FAISS.from_texts(chunks,embeddings)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

template = """
        Vous êtes un agent IA spécialisé pour répondre aux questions sur RA.
        Expliquez le concept ou répondez à la question concernant RA.
        Pour créer la réponse, veuillez utiliser uniquement les informations fournies dans le contexte (Context).
        Répondez avec des mots simples.
        Si nécessaire, incluez également des explications.
        Répondez seulement en français
        Question: {question}
        Context: {context}
        Réponse:
"""

prompt = PromptTemplate.from_template(template)

In [ ]:
retriever_faiss = vectorstore.as_retriever(search_kwargs={"k":6})

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = (
    {
        "context": itemgetter("question") | retriever_faiss,
        "question": itemgetter("question"),
    }
    | prompt
    | model1
    | parser
)

In [ ]:
qst = "c'est quoi la realité augmente?"
print(f"Answer: {chain.invoke({'question': qst})}")

Answer: La réalité augmentée (RA) est une technique qui permet de combiner des informations numériques (images virtuelles) avec des images réelles (capturées en temps réel) pour enrichir l'expérience utilisateur.


In [ ]:
qst = "c est quoi RA?"
print(f"Answer: {chain.invoke({'question': qst})}")

Answer: La RA (Réalité Augmentée) est une technique qui permet de combiner des informations numériques (images virtuelles) avec des images réelles (capturées en temps réel) pour enrichir l'expérience utilisateur de la réalité.
